<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Ecorces_Arbres/IdEcorces-ResConv-TFLiteModelMaker-Colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Flower classification with TensorFlow Lite Model Maker with TensorFlow 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">      
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/lite/codelabs/flower_classification/ml/Flower_Classification_with_TFLite_Model_Maker.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

Model Maker library simplifies the process of adapting and converting a TensorFlow neural-network model to particular input data when deploying this model for on-device ML applications.

This notebook shows an end-to-end example that utilizes this Model Maker library to illustrate the adaption and conversion of a commonly-used image classification model to classify flowers on a mobile device.

## Prerequisites

To run this example, we first need to make a copy of this notebook. Click on "Copy to Drive" at the top of this notebook. Then we need to install serveral required packages, including Model Maker package that in github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 4.3 MB/s 
     |████████████████████████████████| 87 kB 8.8 MB/s 
     |████████████████████████████████| 234 kB 61.8 MB/s 
     |████████████████████████████████| 6.4 MB 67.1 MB/s 
     |████████████████████████████████| 1.1 MB 53.9 MB/s 
     |████████████████████████████████| 3.4 MB 55.6 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 1.1 MB 51.5 MB/s 
     |████████████████████████████████| 840 kB 56.4 MB/s 
     |████████████████████████████████| 120 kB 49.5 MB/s 
     |████████████████████████████████| 1.2 MB 53.7 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 99 kB 12.9 MB/s 
     |████████████████████████████████| 47.7 MB 76 kB/s 
     |████████████████████████████████| 352 kB 72.2 MB/s 
     |████████████████████████████████| 462 kB 48.3 MB/s 
     |███████████████

Import the required packages.

In [2]:
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')

import matplotlib.pyplot as plt
import numpy as np

## Simple End-to-End Example

### Get the data path

Let's get some images to play with this simple end-to-end example. Hundreds of images is a good start for Model Maker while more data could achieve better accuracy.

In [ ]:
# image_path = tf.keras.utils.get_file(
#       'flower_photos',
#       'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#       untar=True)

You could replace `image_path` with your own image folders. As for uploading data to colab, you could find the upload button in the left sidebar shown in the image below with the red rectangle. Just have a try to upload a zip file and unzip it. The root file path is the current path.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your images to the cloud, you could try to run the library locally following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) in github.

In [1]:
dict_arbres = {
    'BOJ' : "Betula alleghaniensis - Bouleau jaune - Yellow birch",
    'BOP' : "Betula papyrifera - Bouleau à papier - White birch",
    'CHR' : "Quercus rubra - Chêne rouge - Northern red oak",
    'EPB' : "Picea glauca - Épinette blanche - White spruce",
    'EPN' : " Picea mariana - Épinette noire - Black spruce",
    'EPO' : "Picea abies - Épinette de Norvège - Norway spruce",
    'EPR' : "Picea rubens - Épinette rouge - Red spruce",
    'ERB' : "Acer platanoides - Érable de Norvège - Norway maple",
    'ERR' : "Acer rubrum - Érable rouge - Red maple",
    'ERS' : "Acer saccharum - Érable à sucre - Sugar maple",
    'FRA' : "Fraxinus americana - Frêne d'Amérique - White ash",
    'HEG' : "Fagus grandifolia - Hêtre à grandes feuilles - American beech",
    'MEL' : "Larix laricina - Mélèze - Tamarack",
    'ORA' : "Ulmus americana - Orme d'Amérique - American elm",
    'OSV' : "Ostrya virginiana - Ostryer de Virginie - American hophornbeam",
    'PEG' : "Populus grandidentata - Peuplier à grandes dents - Big-tooth aspen",
    'PET' : "Populus tremuloides - Peuplier faux tremble - Quaking aspen",
    'PIB' : "Pinus strobus - Pin blanc - Eastern white pine",
    'PID' : "Pinus rigida - Pin rigide - Pitch pine",
    'PIR' : "Pinus resinosa - Pin rouge - Red pine",
    'PRU' : "Tsuga canadensis - Pruche du Canada - Eastern Hemlock",
    'SAB' : "Abies balsamea - Sapin Baumier - Balsam fir",
    'THO' : "Thuja occidentalis - Thuya occidental - Northern white cedar",
}

print("Dictionnaire mémorisé")

Dictionnaire mémorisé


### Création des répertoires de données

Nous allons créer un répertoire de base `donnees`, un répertoire `lab_ecorces` où les données seront réparties en données d'entraînement, de validation et de test pour chaque classe cible.

Enfin, un répertoire `modeles` pour mémoriser les modèles une fois entraînés.

In [8]:
try:
    os.mkdir("/content/donnees/")
except OSError:
    pass
try:
    os.mkdir("/content/lab_ecorces/")
except OSError:
    pass
try:
    os.mkdir("/content/modeles/")
except OSError:
    pass


### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>


8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données « barknet » de 32 Go avec la commande suivante:

In [ ]:
# Attention! Jeu réduit de données 1.5 Go - plus rapide à télécharger et à traiter
!kaggle datasets download claudecoulombe/barknet --unzip -p /content/donnees/
repertoire_entree = "/content/donnees/BarkNet"

### Répartition des données

In [11]:
# Installation des bibliothèques Python `split-folders` et `tqdm`
!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement, de validation et de tests
import splitfolders
import pathlib

#### répertoire des données une fois réparties
repertoire_donnees_reparties = "/content/lab_ecorces"
# => train, val, test

nombre_images = len(list(pathlib.Path(repertoire_entree).glob('*/*.jpg')))
print("Nombre total d'images:",nombre_images)

splitfolders.ratio(repertoire_entree, 
                   output=repertoire_donnees_reparties, 
                   seed=42, 
                   ratio = (0.65, 0.15, 0.20)
                   )

print("\nRépartition des données terminée!")

In [23]:
image_path = "/content/lab_ecorces/train/"

### Run the example
The example just consists of 4 lines of code as shown below, each of which representing one step of the overall process.


1.   Load input data specific to an on-device ML app. Split it to training data and testing data.

In [24]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 3981, num_label: 20, labels: BOJ, BOP, CHR, EPB, EPN, EPO, EPR, ERR, ERS, FRA, HEG, MEL, ORA, OSV, PET, PIB, PIR, PRU, SAB, THO.


2. Customize the TensorFlow model.

In [33]:
model = image_classifier.create(train_data,use_augmentation=True,epochs=50,dropout_rate=0.2)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_5 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 1280)              0         
                                                                 
 dense_5 (Dense)             (None, 20)                25620     
                                                                 
Total params: 3,438,644
Trainable params: 25,620
Non-trainable params: 3,413,024
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
111/111 [==============================] - 35s 290ms/step - loss: 2.1394 - accuracy: 0.4158
Epoch 2/50
111/111 [==============================] - 32s 290ms/step - loss: 1.6251 - accuracy: 0.6028
Epoch 3/50
111/111 [==============================] - 32s 291ms/step - loss: 1.5631 - accuracy: 0.6298
Epoch 4/50
111/111 [==============================] - 32s 290ms/step - loss: 1.4994 - accuracy: 0.6613
Epoch 5/50
111/111 [==============================] - 32s 290ms/step - loss: 1.4589 - accuracy: 0.6799
Epoch 6/50
111/111 [==============================] - 32s 291ms/step - loss: 1.4427 - accuracy: 0.6867
Epoch 7/50
111/111 [==============================] - 32s 290ms/step - loss: 1.4250 - accuracy: 0.7066
Epoch 8/50
111/111 [==============================] - 37s 330ms/step - loss: 1.4094 - accuracy: 0.7019
Epoch 9/50
111/111 [==============================] - 38s 337ms/step - loss: 1.3881 - accuracy: 0.7128
Epoch 10/50
111/111 [==============================] - 33s 294ms/step - l

3. Evaluate the model.

In [34]:
loss, accuracy = model.evaluate(test_data)

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


13/13 [==============================] - 27s 283ms/step - loss: 1.1507 - accuracy: 0.8070


In [49]:
# history = model.train(train_data)

In [43]:
print("Exactitude données de test accuracy:   %0.2f" % accuracy)

Exactitude données de test accuracy:   0.81


In [47]:
predictions = model.predict_top_k(test_data)
# predictions_index = np.argmax(predictions, axis=1)
predictions_index = [resultat[0][0] for resultat in predictions]
print(predictions_index)

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


['MEL', 'PRU', 'EPO', 'PIB', 'HEG', 'FRA', 'FRA', 'ERS', 'BOJ', 'ORA', 'MEL', 'MEL', 'EPO', 'EPO', 'EPB', 'FRA', 'EPO', 'ERS', 'SAB', 'ERS', 'PRU', 'ORA', 'CHR', 'BOP', 'BOP', 'FRA', 'FRA', 'ERS', 'CHR', 'ERS', 'ERS', 'BOJ', 'MEL', 'BOP', 'MEL', 'CHR', 'EPB', 'HEG', 'EPR', 'ERS', 'EPO', 'BOJ', 'BOJ', 'BOP', 'ERS', 'FRA', 'BOP', 'OSV', 'EPB', 'THO', 'ERS', 'ERS', 'ERR', 'BOJ', 'MEL', 'ERS', 'CHR', 'FRA', 'CHR', 'EPB', 'MEL', 'SAB', 'ERS', 'BOJ', 'MEL', 'MEL', 'CHR', 'CHR', 'EPR', 'CHR', 'ERR', 'PET', 'ERR', 'EPN', 'SAB', 'ERR', 'BOP', 'CHR', 'ERS', 'THO', 'THO', 'ERR', 'CHR', 'EPR', 'CHR', 'ERR', 'EPO', 'CHR', 'BOJ', 'FRA', 'PIB', 'OSV', 'CHR', 'EPN', 'HEG', 'ERR', 'ORA', 'EPN', 'THO', 'PIB', 'PIR', 'SAB', 'EPR', 'FRA', 'THO', 'EPO', 'FRA', 'EPN', 'SAB', 'CHR', 'HEG', 'MEL', 'CHR', 'FRA', 'PET', 'ORA', 'ERR', 'BOP', 'ERR', 'CHR', 'OSV', 'ERS', 'OSV', 'FRA', 'THO', 'THO', 'OSV', 'PIB', 'MEL', 'ERR', 'EPN', 'ERR', 'HEG', 'ERR', 'PIB', 'BOJ', 'PET', 'MEL', 'CHR', 'ERR', 'EPN', 'THO', 'BOP'

In [54]:
len(predictions_index)

399

In [51]:
test_data = test_data.gen_dataset()

In [55]:
len(test_data)

399

In [67]:
def get_labels_from_tfdataset(tfdataset, batched=False):
    labels = list(map(lambda x: x[1], tfdataset)) # Get labels 
    if not batched:
        return tf.concat(labels, axis=0) # concat the list of batched labels
    return labels

In [80]:
list_no_arbres = get_labels_from_tfdataset(test_data)._numpy().tolist()
print(list_no_arbres)
vraies_etiquettes_index = [dict_no_arbres_ID[str(no_arbres+1)] for no_arbres in list_no_arbres]
print(vraies_etiquettes_index)

[11, 17, 3, 15, 10, 9, 9, 8, 0, 4, 11, 11, 5, 4, 3, 9, 5, 8, 18, 8, 17, 13, 2, 0, 1, 9, 9, 8, 2, 8, 8, 0, 17, 0, 11, 2, 3, 10, 6, 8, 3, 0, 0, 1, 8, 9, 1, 2, 14, 19, 8, 8, 7, 0, 11, 13, 2, 9, 2, 3, 11, 18, 8, 0, 11, 11, 18, 13, 6, 1, 7, 14, 17, 4, 18, 7, 1, 2, 8, 19, 19, 10, 7, 6, 2, 7, 5, 2, 1, 9, 15, 13, 2, 11, 10, 10, 12, 4, 19, 15, 16, 18, 4, 9, 19, 5, 9, 4, 15, 2, 3, 11, 2, 9, 14, 12, 12, 1, 7, 2, 13, 8, 13, 9, 13, 19, 13, 15, 11, 7, 4, 2, 10, 7, 15, 0, 14, 11, 2, 7, 4, 19, 1, 12, 17, 9, 8, 18, 8, 11, 13, 8, 5, 9, 7, 18, 13, 7, 14, 5, 10, 18, 2, 8, 5, 11, 15, 19, 13, 17, 9, 9, 11, 2, 2, 3, 9, 11, 7, 7, 14, 15, 6, 16, 14, 18, 14, 14, 9, 7, 8, 16, 17, 9, 8, 2, 18, 14, 18, 3, 4, 16, 5, 18, 1, 7, 4, 8, 5, 5, 14, 18, 18, 11, 2, 18, 7, 2, 14, 18, 9, 11, 3, 1, 5, 2, 15, 7, 0, 2, 2, 6, 12, 5, 0, 0, 8, 9, 17, 10, 17, 1, 18, 8, 17, 8, 9, 5, 5, 17, 1, 9, 4, 2, 8, 16, 10, 4, 7, 15, 18, 13, 7, 4, 8, 0, 5, 18, 8, 8, 9, 5, 19, 3, 6, 1, 5, 7, 19, 3, 9, 13, 8, 0, 11, 2, 17, 18, 16, 15, 15, 2, 18, 2

In [81]:
from sklearn import metrics
score = metrics.accuracy_score(vraies_etiquettes_index, predictions_index)
print("Exactitude:   %0.2f" % score)

Exactitude:   0.01


In [82]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matrice de confusion',
                          cmap=plt.cm.Blues):
    plt.figure(figsize=(14,12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matrice de confusion normalisée")
    else:
        print('Matrice de confusion non normalisée')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Vraies étiquettes')
    plt.xlabel('Étiquettes prédites')

cm = metrics.confusion_matrix(vraies_etiquettes_index, predictions_index)
class_names = list(train_data.class_names)
plot_confusion_matrix(cm, classes=class_names)

AttributeError: ignored

4.  Export to TensorFlow Lite model.
You could download it in the left sidebar same as the uploading part for your own use.

In [83]:
model.export(export_dir='.')

INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Assets written to: /tmp/tmpmuzghih_/assets


INFO:tensorflow:Assets written to: /tmp/tmpmuzghih_/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp6i4hne_c/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp6i4hne_c/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./ModeleEcorceIA.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./ModeleEcorceIA.tflite


5. Download the trained model by clicking on the folder icon on the left hand side. Right-click on "ModeleEcorceIA.tflite" and select download. Or run the following code:

In [85]:
from google.colab import files
files.download('ModeleEcorceIA.tflite') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

After this simple 5 steps, we can now continue to the next step in the [codelab](https://codelabs.developers.google.com/codelabs/recognize-flowers-with-tensorflow-on-android-beta/#2).

For a more comprehensive guide to TFLite Model Maker, please refer to this [notebook](https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/demo/image_classification.ipynb) and its [documentation](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).